# Time Dependent Q-Learning

In [ ]:
import gym 
import operator
import itertools
import random
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import deque
from random import randint

In [ ]:
env = gym.make('CartPole-v1')
env._max_episode_steps = 5000
number_of_games = 10000000
ACTION_SPACE = env.action_space.n #number of possible actions
OBSERVATION_SPACE = env.observation_space.shape[0] #number of observable variables
STATES_IN_INTERVAL = 20

In [ ]:
def create_state_intervals():
    intervals = np.zeros((OBSERVATION_SPACE, STATES_IN_INTERVAL))
    intervals[0] = np.linspace(-4.8, 4.8, STATES_IN_INTERVAL)
    intervals[1] = np.linspace(-3.5, 3.5, STATES_IN_INTERVAL)
    intervals[2] = np.linspace(-0.42, 0.42, STATES_IN_INTERVAL)
    intervals[3] = np.linspace(-4, 4, STATES_IN_INTERVAL)
    return intervals

In [ ]:
def discretize_observation(observation):
    discrete_observation = np.array([np.digitize(observation[index], INTERVALS[index])-1 for index in range(OBSERVATION_SPACE)])
    # if some value is under the lower border ignore it and give it min value
    discrete_observation = [0 if x<0 else x for x in discrete_observation]
    return discrete_observation

In [ ]:
def get_all_possible_states():
    digits = len(str(STATES_IN_INTERVAL))
    state_indices = [str(state_index).zfill(digits) for state_index in range(STATES_IN_INTERVAL)] # all encodings for a single observation variable
    states = [state_indices for i in range(OBSERVATION_SPACE)] # for each observation variable a list of its encodings
    states = list(itertools.product(*states)) # get all permutation of all state encodings (->list of tuples)
    states = [''.join(x) for x in states] # join tuples to a single string
    return states

In [ ]:
def observation_to_state(observation):
    discrete_observation = discretize_observation(observation)
    digits = len(str(STATES_IN_INTERVAL))
    
    state = ''
    for state_id in discrete_observation:
        if len(str(state_id)) < digits:
            state += str(state_id).zfill(digits)
        else:
            state += str(state_id)
    return state

In [ ]:
def init_q_table(states, actions):
    q_table = dict()
    for state in states:
        q_table[state] = dict()
        for action in actions:
            q_table[state][action] = np.random.randint(10)
    return q_table

In [ ]:
def get_action(q_table, state, alpha):
    action = 0 if q_table[state][0] > q_table[state][1] else 1
    if(random.random() < alpha):
        action += 1 
        action %= 2
        
    return action

In [ ]:
def max_a(q_table, next_state):
    return max(q_table[next_state][k] for k in q_table[next_state].keys())

In [ ]:
def update_q_table(q_table, state, action, next_state, reward, alpha, gamma):
    q_s_a = q_table[state][action]
    q_table[state][action] = q_s_a + alpha * (reward + gamma * max_a(q_table, next_state) - q_s_a)
    return q_table

In [ ]:
INTERVALS = create_state_intervals()

for i in range(1):
    print('EPISODE:', i)
            
    last100_rewards = deque(maxlen=100) # fifo queue
    game_max = []
    game_mean = []
    solved = False
    
    q_table = init_q_table(get_all_possible_states(), [0, 1])
    
    
    alpha = 1
    gamma = 0.9
          
    for game in range(number_of_games):
        
        overall_reward, done = 0, False
        observation = env.reset()
        state = observation_to_state(observation)
        
        if alpha > 0.01:
            alpha *= 0.999
            
        while not done:
            #if game % 1000 == 0: env.render()
                       
            action = get_action(q_table, state, alpha)

            observation, reward, done, _ = env.step(action)
            
            next_state = observation_to_state(observation)
            overall_reward += reward

            if done: reward = -5000 # punish if agent dies
                
            update_q_table(q_table, state, action, next_state, reward, alpha, gamma)

            state = next_state

        if game % 100 == 0 and game != 0:
            print('Episode:', game,  
                  'Mean-Reward:', np.mean(last100_rewards), 
                  'Max-Reward:', max(last100_rewards),
                  'Alpha:', alpha                     
                 )
            game_max.append(max(last100_rewards))
            game_mean.append(np.mean(last100_rewards))
            
        if (np.mean(last100_rewards) >= 195) and not solved: 
            print('TASK COMPLETED LAST 100 GAMES HAD AN AVERAGE SCORE >=195 ON GAME', game)
            print(last100_rewards)
            solved = True
                       
        
        last100_rewards.append(overall_reward) 

EPISODE: 0


C:\Users\bejahrer\AppData\Local\Continuum\miniconda3\envs\ml\lib\site-packages\numpy\core\fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\bejahrer\AppData\Local\Continuum\miniconda3\envs\ml\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Episode: 100 Mean-Reward: 15.28 Max-Reward: 35.0 Alpha: 0.9038873549665959
Episode: 200 Mean-Reward: 12.55 Max-Reward: 38.0 Alpha: 0.8178301806491574
Episode: 300 Mean-Reward: 13.16 Max-Reward: 40.0 Alpha: 0.7399663251239436
Episode: 400 Mean-Reward: 15.28 Max-Reward: 61.0 Alpha: 0.6695157201007336
Episode: 500 Mean-Reward: 16.97 Max-Reward: 55.0 Alpha: 0.6057725659163237
Episode: 600 Mean-Reward: 17.14 Max-Reward: 72.0 Alpha: 0.548098260578011
Episode: 700 Mean-Reward: 22.04 Max-Reward: 62.0 Alpha: 0.4959150020176678
Episode: 800 Mean-Reward: 24.77 Max-Reward: 69.0 Alpha: 0.44869999946146477
Episode: 900 Mean-Reward: 30.46 Max-Reward: 85.0 Alpha: 0.4059802359226587
Episode: 1000 Mean-Reward: 35.96 Max-Reward: 168.0 Alpha: 0.36732772934619257
Episode: 1100 Mean-Reward: 40.5 Max-Reward: 123.0 Alpha: 0.33235524492954527
Episode: 1200 Mean-Reward: 48.28 Max-Reward: 158.0 Alpha: 0.3007124156643058
Episode: 1300 Mean-Reward: 55.03 Max-Reward: 128.0 Alpha: 0.2720822322326576
Episode: 1400 Me

Episode: 9500 Mean-Reward: 220.67 Max-Reward: 378.0 Alpha: 0.009998671593271896
Episode: 9600 Mean-Reward: 232.43 Max-Reward: 443.0 Alpha: 0.009998671593271896
Episode: 9700 Mean-Reward: 231.69 Max-Reward: 337.0 Alpha: 0.009998671593271896
Episode: 9800 Mean-Reward: 228.45 Max-Reward: 419.0 Alpha: 0.009998671593271896
Episode: 9900 Mean-Reward: 233.09 Max-Reward: 451.0 Alpha: 0.009998671593271896
Episode: 10000 Mean-Reward: 251.94 Max-Reward: 431.0 Alpha: 0.009998671593271896
Episode: 10100 Mean-Reward: 252.7 Max-Reward: 392.0 Alpha: 0.009998671593271896
Episode: 10200 Mean-Reward: 256.56 Max-Reward: 404.0 Alpha: 0.009998671593271896
Episode: 10300 Mean-Reward: 246.57 Max-Reward: 346.0 Alpha: 0.009998671593271896
Episode: 10400 Mean-Reward: 250.1 Max-Reward: 426.0 Alpha: 0.009998671593271896
Episode: 10500 Mean-Reward: 238.59 Max-Reward: 407.0 Alpha: 0.009998671593271896
Episode: 10600 Mean-Reward: 228.29 Max-Reward: 332.0 Alpha: 0.009998671593271896
Episode: 10700 Mean-Reward: 241.01 

Episode: 19700 Mean-Reward: 345.26 Max-Reward: 797.0 Alpha: 0.009998671593271896
Episode: 19800 Mean-Reward: 336.55 Max-Reward: 439.0 Alpha: 0.009998671593271896
Episode: 19900 Mean-Reward: 337.98 Max-Reward: 788.0 Alpha: 0.009998671593271896
Episode: 20000 Mean-Reward: 339.43 Max-Reward: 572.0 Alpha: 0.009998671593271896
Episode: 20100 Mean-Reward: 319.19 Max-Reward: 529.0 Alpha: 0.009998671593271896
Episode: 20200 Mean-Reward: 349.61 Max-Reward: 613.0 Alpha: 0.009998671593271896
Episode: 20300 Mean-Reward: 338.02 Max-Reward: 678.0 Alpha: 0.009998671593271896
Episode: 20400 Mean-Reward: 354.18 Max-Reward: 604.0 Alpha: 0.009998671593271896
Episode: 20500 Mean-Reward: 353.92 Max-Reward: 755.0 Alpha: 0.009998671593271896
Episode: 20600 Mean-Reward: 340.26 Max-Reward: 594.0 Alpha: 0.009998671593271896
Episode: 20700 Mean-Reward: 337.01 Max-Reward: 558.0 Alpha: 0.009998671593271896
Episode: 20800 Mean-Reward: 344.0 Max-Reward: 751.0 Alpha: 0.009998671593271896
Episode: 20900 Mean-Reward: 3

Episode: 29900 Mean-Reward: 289.14 Max-Reward: 587.0 Alpha: 0.009998671593271896
Episode: 30000 Mean-Reward: 260.77 Max-Reward: 422.0 Alpha: 0.009998671593271896
Episode: 30100 Mean-Reward: 271.27 Max-Reward: 443.0 Alpha: 0.009998671593271896
Episode: 30200 Mean-Reward: 275.59 Max-Reward: 450.0 Alpha: 0.009998671593271896
Episode: 30300 Mean-Reward: 281.68 Max-Reward: 426.0 Alpha: 0.009998671593271896
Episode: 30400 Mean-Reward: 239.45 Max-Reward: 353.0 Alpha: 0.009998671593271896
Episode: 30500 Mean-Reward: 263.1 Max-Reward: 431.0 Alpha: 0.009998671593271896
Episode: 30600 Mean-Reward: 290.14 Max-Reward: 448.0 Alpha: 0.009998671593271896
Episode: 30700 Mean-Reward: 286.04 Max-Reward: 539.0 Alpha: 0.009998671593271896
Episode: 30800 Mean-Reward: 283.38 Max-Reward: 552.0 Alpha: 0.009998671593271896
Episode: 30900 Mean-Reward: 295.5 Max-Reward: 705.0 Alpha: 0.009998671593271896
Episode: 31000 Mean-Reward: 284.53 Max-Reward: 796.0 Alpha: 0.009998671593271896
Episode: 31100 Mean-Reward: 27

Episode: 40100 Mean-Reward: 287.21 Max-Reward: 510.0 Alpha: 0.009998671593271896
Episode: 40200 Mean-Reward: 279.35 Max-Reward: 571.0 Alpha: 0.009998671593271896
Episode: 40300 Mean-Reward: 294.67 Max-Reward: 684.0 Alpha: 0.009998671593271896
Episode: 40400 Mean-Reward: 285.17 Max-Reward: 561.0 Alpha: 0.009998671593271896
Episode: 40500 Mean-Reward: 290.33 Max-Reward: 468.0 Alpha: 0.009998671593271896
Episode: 40600 Mean-Reward: 301.87 Max-Reward: 658.0 Alpha: 0.009998671593271896
Episode: 40700 Mean-Reward: 293.63 Max-Reward: 780.0 Alpha: 0.009998671593271896
Episode: 40800 Mean-Reward: 284.83 Max-Reward: 558.0 Alpha: 0.009998671593271896
Episode: 40900 Mean-Reward: 312.57 Max-Reward: 591.0 Alpha: 0.009998671593271896
Episode: 41000 Mean-Reward: 370.8 Max-Reward: 1109.0 Alpha: 0.009998671593271896
Episode: 41100 Mean-Reward: 324.08 Max-Reward: 766.0 Alpha: 0.009998671593271896
Episode: 41200 Mean-Reward: 380.42 Max-Reward: 1022.0 Alpha: 0.009998671593271896
Episode: 41300 Mean-Reward: